In [ ]:
# @title
!pip install gensim

In [ ]:
%pip install pyLDAvis

In [ ]:
%pip install "numpy<2"
%pip install spacy
spacy.cli.download("en_core_web_sm")

In [ ]:
# -*- coding: utf-8 -*-
"""
Created on Tue Jul 29 13:54:48 2025

@author: dnd2129
NLP processes
"""

import pandas as pd

import numpy as np

import re
import nltk
nltk.download('stopwords')
nltk.download('punkt')
from nltk.corpus import stopwords
from nltk import punkt
from nltk import word_tokenize, ngrams
from nltk.util import ngrams

import spacy

import gensim
# Sklearn
from sklearn.decomposition import LatentDirichletAllocation, TruncatedSVD
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import GridSearchCV
from pprint import pprint
# Plotting tools

#import pyLDAvis
#import pyLDAvis.sklearn
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
from google.colab import auth
import gspread
from google.auth import default

In [ ]:
auth.authenticate_user()
creds, _ = default()
gc = gspread.authorize(creds)

In [1]:
df = pd.read_excel("/content/drive/MyDrive/CBS/CBS-Github-DND/Student Cluster Analysis and LDA-20250714T173604Z-1-001/Student Cluster Analysis and LDA/raw data/Project - Student Course Clustering/Student Course Elective Enrollments Graduates 2016-2025.xlsx")
df.head()
#unq_courses= df.combined_name.unique()



NameError: name 'pd' is not defined

#NLP Process

## Cleaning Text

In [ ]:

#cleaning texts
df['clean_name'] = df['combined_name'].fillna('') #changing nan to blank
df['clean_name'] = df['clean_name'].str.replace(r'\s+', ' ', regex=True)#cleaning new line chars
df['clean_name'] = df['clean_name'].str.replace("'", '', regex=False)#removing single quotes

df.head()


### Remove stopwords

In [ ]:
def rem_sw(var_in):
    sw = list(set(stopwords.words('english')))
    tmp = [word for word in var_in.split() if word not in sw]
    return ' '.join(tmp)

In [ ]:
df['clean_name'] = df['clean_name'].apply(rem_sw)
df.head()

In [ ]:
df.tail()

## n-gram extraction

### Manual Tokenizing::: may be ignore as tfidf may be better!

In [ ]:
'''
Tokenize each sentence into a list of words, removing punctuations and
unnecessary characters altogether.

Using spaCy because nltk wasn't working-- and according to article, spaCy is better anyway....
'''
nlp = spacy.load("en_core_web_sm")

def generate_ngrams_spacy(text):
    doc = nlp(text.lower())
    tokens = [token.text for token in doc if not token.is_punct and not token.is_space]

    unigrams = list(zip(tokens))
    bigrams = list(zip(tokens, tokens[1:])) if len(tokens) > 1 else []
    trigrams = list(zip(tokens, tokens[1:], tokens[2:])) if len(tokens) > 2 else []

    return unigrams, bigrams, trigrams


In [ ]:
df[['unigrams', 'bigrams', 'trigrams']] = df['clean_name'].fillna('').apply(
    lambda x: pd.Series(generate_ngrams_spacy(x))
)

In [ ]:
df.head()

In [ ]:
#df.to_csv('course names uni-bi-trigrams.csv')

In [ ]:
subset = df[['clean_name', 'unigrams', 'bigrams', 'trigrams']].copy()

# Convert unhashable list columns to strings
subset['unigrams'] = subset['unigrams'].astype(str)
subset['bigrams'] = subset['bigrams'].astype(str)
subset['trigrams'] = subset['trigrams'].astype(str)

# Now drop duplicates safely
unique_subset = subset.drop_duplicates()

# Check it
unique_subset.head()


In [ ]:
#unique_subset.to_csv("unique_subset.csv", index=False)

Manually looking at unigrams, bigrams and trigrams to decide which to use for LDA

### Tf-idf

In [ ]:
# Your existing setup
vectorizer = TfidfVectorizer(ngram_range=(1, 3))
X = vectorizer.fit_transform(df['clean_name'])

# Sum TF-IDF values across all documents
tfidf_sums = X.sum(axis=0).A1  # A1 flattens the matrix to a 1D array
feature_names = vectorizer.get_feature_names_out()

gives feature names, i.e., ngrams

getting  SCORE FOR EACH ngram

In [2]:
# Create DataFrame of words and scores
tfidf_scores = pd.DataFrame({
    'term': feature_names,
    'score': tfidf_sums
})

# Sort by score descending
tfidf_scores = tfidf_scores.sort_values(by='score', ascending=False)

# View top words/phrases
print(tfidf_scores.head(30))

NameError: name 'pd' is not defined

write csv

In [ ]:
tfidf_scores.to_csv("tfidf_scores ngrams 1,2,3.csv", index=False)

### IDF: maybe this will make it easier to sort out words!
higher scores--> less frequent-- may be more valuable to us

In [ ]:
import math
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer

# Step 1: Use CountVectorizer to get n-grams (1 to 3)
vectorizer = CountVectorizer(ngram_range=(1, 3), analyzer='word')
X = vectorizer.fit_transform(df['clean_name'])

# Step 2: Extract n-gram feature names
feature_names = vectorizer.get_feature_names_out()

# Step 3: Get number of documents each n-gram appears in (document frequency)-- I think this is counting each ngram list as a document, and NOT the clean course name.
# This is a binary presence matrix — whether term appears in document
document_frequencies = (X > 0).sum(axis=0).A1  # .A1 flattens to 1D array
num_documents = X.shape[0]


In [ ]:
print(feature_names)

In [ ]:
# Step 4: Compute smoothed IDF scores
idf_scores = {
    term: math.log((1 + num_documents) / (1 + dfreq)) + 1
    for term, dfreq in zip(feature_names, document_frequencies)
}

In [ ]:
# Step 5: Convert to DataFrame and sort
idf_df = pd.DataFrame(idf_scores.items(), columns=["term", "idf_score"])
idf_df['total_count_across_ngrams'] = document_frequencies #adding number of documents(coursename) each term appears in
#matches substring and \b ..\b matches word boundaries... so 'database' won't count in 'data'.
#This is re-countign even if it occurs in shorter/bigger ngrams for the same name--- need to get just count.
idf_df = idf_df.sort_values(by="idf_score", ascending=False)

idf_df.head(20)



#### Add count of unique course_identifiers which have each term

 Re-doing clean name with ngrams list linked to clean names using ccount vetorizer instead of manual spacy command like under 'manual tokenization'

In [ ]:
def generate_ngrams_countvectorizer(text):
  # Combine into a list since CountVectorizer expects an iterable of docs
  docs = [text.lower()]

  # Unigrams
  vectorizer_uni = CountVectorizer(ngram_range=(1, 1))
  unigrams = vectorizer_uni.fit(docs)
  unigrams_list = list(unigrams.vocabulary_.keys())

  # Bigrams
  vectorizer_bi = CountVectorizer(ngram_range=(2, 2))
  bigrams = vectorizer_bi.fit(docs)
  bigrams_list = list(bigrams.vocabulary_.keys())

  # Trigrams
  vectorizer_tri = CountVectorizer(ngram_range=(3, 3))
  trigrams = vectorizer_tri.fit(docs)
  trigrams_list = list(trigrams.vocabulary_.keys())

  return unigrams_list, bigrams_list, trigrams_list
# use
df[['unigrams', 'bigrams', 'trigrams']] = df['clean_name'].fillna('').apply(
    lambda x: pd.Series(generate_ngrams_spacy(x))
)

In [ ]:
df.head()

In [ ]:
subset = df[['clean_name', 'unigrams', 'bigrams', 'trigrams']].copy()

# Convert unhashable list columns to strings
subset['unigrams'] = subset['unigrams'].astype(str)
subset['bigrams'] = subset['bigrams'].astype(str)
subset['trigrams'] = subset['trigrams'].astype(str)

# Now drop duplicates safely
unique_subset = subset.drop_duplicates()

# Check it
unique_subset.head()


In [3]:
import ast

unique_subset['unigrams'] = unique_subset['unigrams'].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else x)
unique_subset['bigrams'] = unique_subset['bigrams'].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else x)
unique_subset['trigrams'] = unique_subset['trigrams'].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else x)


NameError: name 'unique_subset' is not defined

In [ ]:
def flatten_and_join(row):
    def tuple_to_str(t):
        if isinstance(t, (tuple, list)):
            return " ".join(t)
        elif isinstance(t, str):
            return t
        else:
            return ""

    def safe_get(col):
        val = row[col]
        if isinstance(val, list):
            return val
        else:
            return []

    unigrams = safe_get('unigrams')
    bigrams = safe_get('bigrams')
    trigrams = safe_get('trigrams')

    all_ngrams = [tuple_to_str(t) for t in (unigrams + bigrams + trigrams) if t]
    return ", ".join(all_ngrams)

unique_subset['all_ngrams'] = unique_subset.apply(flatten_and_join, axis=1)

unique_subset.head()

In [ ]:
# Step 3: Split combined n-grams into individual terms
unique_subset['all_ngrams'] = unique_subset['all_ngrams'].apply(lambda x: [t.strip() for t in x.split(',') if t.strip()])
unique_subset.head()

In [ ]:
#Just for quick future use.
#unique_subset.to_csv('/content/drive/MyDrive/CBS/CBS-Github-DND/Student Cluster Analysis and LDA-20250714T173604Z-1-001/Student Cluster Analysis and LDA/Student Cluster Analysis GDrive/CV_unique uni-bi-trigram_subset.csv')

In [ ]:

# Step 4: Flatten into a single list of terms
all_terms = [term for terms_list in unique_subset['all_ngrams'] for term in terms_list]

# Step 5: Count term occurrences
term_counts = pd.Series(all_terms).value_counts().reset_index()
term_counts.columns = ['term', 'count']
term_counts.head()

In [ ]:
term_counts.tail()

In [ ]:
# Step 6: Merge counts back into your idf_df
idf_with_counts = idf_df.merge(term_counts, on='term', how='left')
idf_with_counts.head()

In [ ]:
idf_with_counts.tail(10)

In [ ]:
# Step 6: Export to CSV
idf_with_counts.to_csv("/content/drive/MyDrive/CBS/CBS-Github-DND/Student Cluster Analysis and LDA-20250714T173604Z-1-001/Student Cluster Analysis and LDA/Student Cluster Analysis GDrive/idf_ngram_COUNT.csv", index=False)



#### What if i remove verbs?

In [ ]:
from nltk.tag import pos_tag

import nltk
nltk.download('averaged_perceptron_tagger_eng')

In [ ]:
tagged_tokens = pos_tag(idf_with_counts['term'])

In [ ]:
idf_with_counts["non_verb_tokens"] = [
    word if not tag.startswith('VB') else ''  # or None if you prefer
    for word, tag in tagged_tokens
]
idf_with_counts.head()

### RAKE/YAKE/ Keyword extraction

#### RAKE (Rapid Automatic Keyword extraction)

In [ ]:
%pip install rake-nltk


In [4]:
nltk.download('punkt_tab')

NameError: name 'nltk' is not defined

In [ ]:
from rake_nltk import Rake

rake = Rake()

def extract_rake_keywords(text):
    rake.extract_keywords_from_text(text)
    return rake.get_ranked_phrases()

df["rake_keywords"] = df["clean_name"].apply(extract_rake_keywords)
df.head()

#### YAKE (Yet Another Keyword Extractor)

In [ ]:
%pip install yake

In [ ]:
import yake
#YAKE gives all words in ranked order with scores to highlight important keywords-- I just want the top 5 keywords (for simplicity) and i dont care about the score.

kw_extractor = yake.KeywordExtractor(top=5)  # limit to top 5 keywords

def extract_top5_yake_keywords(text):
    keywords = kw_extractor.extract_keywords(text)  # list of (keyword, score) tuples
    # Extract only the keywords (ignore scores)
    return [kw for kw, score in keywords]

df["yake_top5_keywords"] = df["clean_name"].apply(extract_top5_yake_keywords)

In [ ]:
df["yake_top5_keywords"].iloc[0]

In [ ]:
df.to_csv("/content/drive/MyDrive/CBS/CBS-Github-DND/Student Cluster Analysis and LDA-20250714T173604Z-1-001/Student Cluster Analysis and LDA/Student Cluster Analysis GDrive/ngrams_rake_yake_keywords.csv")

# LDA processes

now, combine all courses for each student into one row so we have one row per studnt. Then,tokenize it like I did above (choose uni/bi or trigrams depending of which worked best).
THEN: only retain the tokens that I manually chose to use from the manual mapping.

### Grouping by student
One row per student-- adding a course_list column which contains all courses they have taken.

In [ ]:
'''
Check the Sparsicity
'''
# Materialize the sparse data
data_dense = data_vectorized.todense()
# Compute Sparsicity = Percentage of Non-Zero cells
print("Sparsicity: ", ((data_dense > 0).sum()/data_dense.size)*100, "%")
#Sparsicity:  0.6425770811326413 %

In [ ]:
#One row per student-- adding a course_list column which contains all courses they have taken.
grouped = df.groupby('uni')